In [17]:
import json
import random
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [18]:


# Open the JSON file and load its contents into a dictionary
with open('games.json', 'r') as f:
    games_dict = json.load(f)
    

# Print the dictionary to verify that it was loaded correctly
print(games_dict)

{'Action Game': ['Grand Theft Auto V', 'Call of Duty: Modern Warfare', "Assassin's Creed Valhalla", 'Doom Eternal', 'Metal Gear Solid V: The Phantom Pain', 'Hades', 'Control', 'Nier: Automata', 'Batman: Arkham Knight', 'Devil May Cry 5', 'Resident Evil Village', 'Deathloop', 'Far Cry 6', "Ghost of Tsushima Director's Cut", 'Returnal'], 'Adventure Game': ['The Legend of Zelda: Breath of the Wild', "Uncharted 4: A Thief's End", 'Red Dead Redemption 2', 'The Last of Us Part II', 'Death Stranding', 'Horizon Zero Dawn', "Assassin's Creed Odyssey", 'God of War', 'Tomb Raider', 'Shadow of the Colossus', 'It Takes Two', 'Kena: Bridge of Spirits', 'Life is Strange: True Colors', 'Psychonauts 2', 'The Artful Escape'], 'Role-Playing Game': ['The Witcher 3: Wild Hunt', 'Skyrim', 'Fallout 4', 'Mass Effect 2', 'Final Fantasy VII Remake', 'Persona 5', 'Dragon Age: Inquisition', 'Divinity: Original Sin II', 'Dark Souls III', 'Kingdom Come: Deliverance', 'Tales of Arise', 'Scarlet Nexus', 'Disco Elysiu

In [19]:
x='Action Game'
games_dict[x]

['Grand Theft Auto V',
 'Call of Duty: Modern Warfare',
 "Assassin's Creed Valhalla",
 'Doom Eternal',
 'Metal Gear Solid V: The Phantom Pain',
 'Hades',
 'Control',
 'Nier: Automata',
 'Batman: Arkham Knight',
 'Devil May Cry 5',
 'Resident Evil Village',
 'Deathloop',
 'Far Cry 6',
 "Ghost of Tsushima Director's Cut",
 'Returnal']

In [31]:

def create_transaction_data(num_transactions, max_games_per_transaction,mode):
    data = []

    game_names = [
        'Action Game', 'Adventure Game', 'Role-Playing Game', 'Strategy Game',
        'Simulation Game', 'Sports Game', 'Puzzle Game', 'Racing Game',
        'Shooter Game', 'Platformer Game', 'Indie Game', 'Horror Game'
    ]

    for transaction_id in range(1, num_transactions + 1):
        if max_games_per_transaction <= 2:
            max_games_per_transaction = 2
        num_games = random.randint(2, max_games_per_transaction)
        games = random.sample(game_names, num_games)
        amount = round(random.uniform(10, 100), 2)
        purchase_date = f'2023-09-{random.randint(1, 30)}'
        discount = round(random.uniform(0, 0.3), 2)
        mpay=random.choice(['cash','card','robbery','UPI','Membership'])


        transaction = {
            'TransactionID': transaction_id,
            'Games': games,
            'Amount': amount,
            'PurchaseDate': purchase_date,
            'Discount': discount,
            'Payment_mode': mpay
            
        }

        data.append(transaction)

    # Convert the data list to JSON format
    json_data = json.dumps(data, indent=4)

    # Save the JSON data to a file
    with open('transaction_data.json', 'w') as json_file:
        json_file.write(json_data)
    
    if mode==1:
        return data
    else:
        return json_data

# Example usage:
# transaction_data = create_transaction_data(10, 5)
# print(transaction_data)

In [53]:

try:

    num_transactions = 1000 #int(input("Enter the number of transactions: "))
    max_games_per_transaction = 12#int(input("Enter the maximum number of games per transaction: "))
    user_game = input("Enter a video game: ")
    if user_game=='1':
        user_game='Action Game'
    num_recommendations = int(input("Enter the number of game recommendations: "))


    transaction_data = create_transaction_data(num_transactions, max_games_per_transaction,1)


    df = pd.DataFrame(transaction_data)

    oht = pd.get_dummies(df['Games'].apply(pd.Series).stack()).groupby(level=0).sum()



    frequent_itemsets = apriori(oht, min_support=0.35, use_colnames=True)

    rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)


    user_rules = rules[rules['antecedents'] == {user_game}]
    user_rules = user_rules.sort_values(by='lift', ascending=False).head(num_recommendations)


    # recommended_games = list(user_rules['consequents'].values)

    
    recommended_games = [list(game)[0] for game in user_rules['consequents']]
    print(f"Recommended games for {user_game}: {recommended_games[0]}")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Recommended games for Action Game: Adventure Game


c:\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
